# Notebook 04: Análisis Geoestadístico

Este notebook implementa:
1. Semivariogramas experimentales
2. Ajuste de modelos teóricos
3. Interpolación Kriging vs IDW
4. Validación cruzada
5. Mapas de predicción con incertidumbre

In [ ]:
# Importar librerías
import sys
sys.path.append('../scripts')

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# Importar módulos propios
from geostatistics import (
    calculate_empirical_variogram,
    fit_variogram_model,
    idw_interpolation,
    ordinary_kriging,
    leave_one_out_cv,
    create_prediction_grid,
    VARIOGRAM_MODELS
)

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Librerías cargadas correctamente")

## 1. Cargar Datos

In [ ]:
# Rutas de datos
DATA_PATH = Path('../data/raw/isla_de_pascua')
PROCESSED_PATH = Path('../data/processed')
OUTPUT_PATH = Path('../outputs/figures')

# Cargar datos
buildings = gpd.read_file(DATA_PATH / 'isla_de_pascua_buildings.geojson')
boundary = gpd.read_file(DATA_PATH / 'isla_de_pascua_boundary.geojson')

print(f"Edificios: {len(buildings)}")
print(f"CRS original: {buildings.crs}")

In [ ]:
# Reproyectar a UTM para trabajar en metros
# Isla de Pascua: UTM zone 12S (EPSG:32712)
buildings_utm = buildings.to_crs(epsg=32712)
boundary_utm = boundary.to_crs(epsg=32712)

# Calcular área de edificios
buildings_utm['area_m2'] = buildings_utm.geometry.area
buildings_utm = buildings_utm[buildings_utm['area_m2'] > 10]  # Filtrar muy pequeños

print(f"Edificios filtrados: {len(buildings_utm)}")
print(f"Área total: {buildings_utm['area_m2'].sum():.0f} m²")

In [ ]:
# Crear grilla de análisis
cell_size = 200  # metros
minx, miny, maxx, maxy = boundary_utm.total_bounds

from shapely.geometry import box

cells = []
for x in np.arange(minx, maxx, cell_size):
    for y in np.arange(miny, maxy, cell_size):
        cells.append(box(x, y, x + cell_size, y + cell_size))

grid = gpd.GeoDataFrame({'geometry': cells}, crs=buildings_utm.crs)
grid = grid[grid.intersects(boundary_utm.unary_union)].reset_index(drop=True)

# Calcular densidad de edificios en cada celda
grid['building_count'] = 0
grid['total_area'] = 0.0

for idx, cell in grid.iterrows():
    buildings_in_cell = buildings_utm[buildings_utm.intersects(cell.geometry)]
    grid.loc[idx, 'building_count'] = len(buildings_in_cell)
    grid.loc[idx, 'total_area'] = buildings_in_cell['area_m2'].sum()

# Filtrar celdas con datos
grid_with_data = grid[grid['building_count'] > 0].copy().reset_index(drop=True)

print(f"Celdas totales: {len(grid)}")
print(f"Celdas con edificios: {len(grid_with_data)}")

## 2. Semivariograma Experimental

In [ ]:
# Extraer coordenadas y valores
coords = np.column_stack([
    grid_with_data.geometry.centroid.x,
    grid_with_data.geometry.centroid.y
])

values = grid_with_data['building_count'].values.astype(float)

print(f"Puntos de muestreo: {len(coords)}")
print(f"Rango de valores: {values.min():.0f} - {values.max():.0f}")
print(f"Media: {values.mean():.2f}")
print(f"Varianza: {values.var():.2f}")

In [ ]:
# Calcular semivariograma empírico
lag_centers, semivariance, n_pairs = calculate_empirical_variogram(
    coords, values, n_lags=15, max_lag=3000
)

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Semivariograma
ax1 = axes[0]
ax1.scatter(lag_centers, semivariance, s=80, c='steelblue', edgecolor='white', linewidth=2)
ax1.set_xlabel('Distancia (m)', fontsize=12)
ax1.set_ylabel('Semivarianza', fontsize=12)
ax1.set_title('Semivariograma Experimental - Densidad de Edificios', fontsize=14)
ax1.grid(True, alpha=0.3)

# Número de pares por lag
ax2 = axes[1]
ax2.bar(lag_centers, n_pairs, width=lag_centers[1]-lag_centers[0], 
        color='coral', edgecolor='white', alpha=0.8)
ax2.set_xlabel('Distancia (m)', fontsize=12)
ax2.set_ylabel('Número de Pares', fontsize=12)
ax2.set_title('Pares por Clase de Distancia', fontsize=14)

plt.tight_layout()
plt.savefig(OUTPUT_PATH / 'variogram_experimental.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Ajuste de Modelos Teóricos

In [ ]:
# Ajustar diferentes modelos
models = {}
model_names = ['spherical', 'exponential', 'gaussian']

for name in model_names:
    models[name] = fit_variogram_model(lag_centers, semivariance, name)
    print(f"\n{name.upper()}:")
    print(f"  Nugget: {models[name]['nugget']:.2f}")
    print(f"  Sill: {models[name]['sill']:.2f}")
    print(f"  Range: {models[name]['range']:.0f} m")
    print(f"  R²: {models[name]['r2']:.4f}")

In [ ]:
# Visualizar ajustes
fig, ax = plt.subplots(figsize=(10, 6))

# Puntos empíricos
ax.scatter(lag_centers, semivariance, s=100, c='black', label='Empírico', zorder=5)

# Líneas de modelos
h_fine = np.linspace(0, lag_centers.max(), 100)
colors = ['#e41a1c', '#377eb8', '#4daf4a']

for (name, model), color in zip(models.items(), colors):
    if 'function' in model:
        y_pred = model['function'](h_fine)
        ax.plot(h_fine, y_pred, '-', linewidth=2, color=color,
                label=f"{name.capitalize()} (R²={model['r2']:.3f})")

ax.set_xlabel('Distancia (m)', fontsize=12)
ax.set_ylabel('Semivarianza', fontsize=12)
ax.set_title('Ajuste de Modelos de Variograma', fontsize=14)
ax.legend(loc='lower right', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_PATH / 'variogram_models_fit.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Seleccionar mejor modelo
best_model_name = max(models, key=lambda x: models[x]['r2'])
best_model = models[best_model_name]

print(f"\n{'='*50}")
print(f"MEJOR MODELO: {best_model_name.upper()}")
print(f"{'='*50}")
print(f"Nugget: {best_model['nugget']:.2f}")
print(f"Sill: {best_model['sill']:.2f}")
print(f"Range: {best_model['range']:.0f} m")
print(f"R²: {best_model['r2']:.4f}")

## 4. Interpolación: Kriging vs IDW

In [ ]:
# Crear grilla de predicción
pred_resolution = 100  # metros
pred_coords = create_prediction_grid(boundary_utm, resolution=pred_resolution)

print(f"Puntos de predicción: {len(pred_coords)}")

In [ ]:
# Interpolación IDW
print("Calculando IDW...")
idw_predictions = idw_interpolation(coords, values, pred_coords, power=2)

print(f"IDW completado: rango {idw_predictions.min():.2f} - {idw_predictions.max():.2f}")

In [ ]:
# Interpolación Kriging
print("Calculando Kriging Ordinario...")

if 'function' in best_model:
    kriging_predictions, kriging_variance = ordinary_kriging(
        coords, values, pred_coords,
        best_model['function'],
        best_model['nugget']
    )
    print(f"Kriging completado: rango {kriging_predictions.min():.2f} - {kriging_predictions.max():.2f}")
    print(f"Varianza de Kriging: {kriging_variance.min():.2f} - {kriging_variance.max():.2f}")
else:
    print("Error: no se pudo ajustar el modelo de variograma")
    kriging_predictions = idw_predictions.copy()
    kriging_variance = np.zeros_like(kriging_predictions)

In [ ]:
# Crear GeoDataFrames para visualización
from shapely.geometry import Point

pred_gdf = gpd.GeoDataFrame({
    'geometry': [Point(x, y) for x, y in pred_coords],
    'idw': idw_predictions,
    'kriging': kriging_predictions,
    'kriging_var': kriging_variance
}, crs=buildings_utm.crs)

In [ ]:
# Visualizar comparación IDW vs Kriging
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# IDW
ax1 = axes[0]
boundary_utm.plot(ax=ax1, facecolor='none', edgecolor='black', linewidth=1)
scatter1 = ax1.scatter(pred_coords[:, 0], pred_coords[:, 1], 
                       c=idw_predictions, cmap='YlOrRd', s=10, alpha=0.7)
plt.colorbar(scatter1, ax=ax1, label='Densidad')
ax1.set_title('IDW (power=2)', fontsize=14)
ax1.set_axis_off()

# Kriging
ax2 = axes[1]
boundary_utm.plot(ax=ax2, facecolor='none', edgecolor='black', linewidth=1)
scatter2 = ax2.scatter(pred_coords[:, 0], pred_coords[:, 1], 
                       c=kriging_predictions, cmap='YlOrRd', s=10, alpha=0.7)
plt.colorbar(scatter2, ax=ax2, label='Densidad')
ax2.set_title(f'Kriging Ordinario ({best_model_name})', fontsize=14)
ax2.set_axis_off()

# Varianza de Kriging (incertidumbre)
ax3 = axes[2]
boundary_utm.plot(ax=ax3, facecolor='none', edgecolor='black', linewidth=1)
scatter3 = ax3.scatter(pred_coords[:, 0], pred_coords[:, 1], 
                       c=kriging_variance, cmap='Blues', s=10, alpha=0.7)
plt.colorbar(scatter3, ax=ax3, label='Varianza')
ax3.set_title('Incertidumbre (Varianza de Kriging)', fontsize=14)
ax3.set_axis_off()

plt.suptitle('Comparación de Métodos de Interpolación', fontsize=16, y=1.02)
plt.tight_layout()
plt.savefig(OUTPUT_PATH / 'interpolation_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Validación Cruzada Leave-One-Out

In [ ]:
# Validación cruzada IDW
print("Validación cruzada IDW...")
cv_idw = leave_one_out_cv(coords, values, method='idw', idw_power=2)

print(f"\nIDW Resultados:")
print(f"  MAE: {cv_idw['MAE']:.3f}")
print(f"  RMSE: {cv_idw['RMSE']:.3f}")
print(f"  ME (sesgo): {cv_idw['ME']:.3f}")
print(f"  R²: {cv_idw['R2']:.4f}")

In [ ]:
# Validación cruzada Kriging
print("Validación cruzada Kriging...")
cv_kriging = leave_one_out_cv(
    coords, values, method='kriging', 
    variogram_params=best_model
)

print(f"\nKriging Resultados:")
print(f"  MAE: {cv_kriging['MAE']:.3f}")
print(f"  RMSE: {cv_kriging['RMSE']:.3f}")
print(f"  ME (sesgo): {cv_kriging['ME']:.3f}")
print(f"  R²: {cv_kriging['R2']:.4f}")

In [ ]:
# Tabla comparativa
cv_results = pd.DataFrame({
    'Método': ['IDW (p=2)', f'Kriging ({best_model_name})'],
    'MAE': [cv_idw['MAE'], cv_kriging['MAE']],
    'RMSE': [cv_idw['RMSE'], cv_kriging['RMSE']],
    'ME (bias)': [cv_idw['ME'], cv_kriging['ME']],
    'R²': [cv_idw['R2'], cv_kriging['R2']]
})

print("\n" + "="*60)
print("RESUMEN VALIDACIÓN CRUZADA")
print("="*60)
print(cv_results.to_string(index=False))

In [ ]:
# Gráficos de validación
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Observado vs Predicho - IDW
ax1 = axes[0, 0]
ax1.scatter(values, cv_idw['predictions'], alpha=0.6, c='steelblue')
ax1.plot([values.min(), values.max()], [values.min(), values.max()], 'r--', linewidth=2)
ax1.set_xlabel('Observado')
ax1.set_ylabel('Predicho')
ax1.set_title(f'IDW: Observado vs Predicho (R²={cv_idw["R2"]:.3f})')

# Observado vs Predicho - Kriging
ax2 = axes[0, 1]
ax2.scatter(values, cv_kriging['predictions'], alpha=0.6, c='coral')
ax2.plot([values.min(), values.max()], [values.min(), values.max()], 'r--', linewidth=2)
ax2.set_xlabel('Observado')
ax2.set_ylabel('Predicho')
ax2.set_title(f'Kriging: Observado vs Predicho (R²={cv_kriging["R2"]:.3f})')

# Distribución de errores - IDW
ax3 = axes[1, 0]
ax3.hist(cv_idw['errors'], bins=20, color='steelblue', alpha=0.7, edgecolor='white')
ax3.axvline(0, color='red', linestyle='--', linewidth=2)
ax3.set_xlabel('Error')
ax3.set_ylabel('Frecuencia')
ax3.set_title(f'Distribución de Errores IDW (ME={cv_idw["ME"]:.3f})')

# Distribución de errores - Kriging
ax4 = axes[1, 1]
ax4.hist(cv_kriging['errors'], bins=20, color='coral', alpha=0.7, edgecolor='white')
ax4.axvline(0, color='red', linestyle='--', linewidth=2)
ax4.set_xlabel('Error')
ax4.set_ylabel('Frecuencia')
ax4.set_title(f'Distribución de Errores Kriging (ME={cv_kriging["ME"]:.3f})')

plt.tight_layout()
plt.savefig(OUTPUT_PATH / 'cross_validation_results.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Conclusiones

In [ ]:
print("="*60)
print("RESUMEN DEL ANÁLISIS GEOESTADÍSTICO")
print("="*60)

print(f"\n1. SEMIVARIOGRAMA:")
print(f"   - Variable analizada: Densidad de edificios por celda")
print(f"   - Mejor modelo: {best_model_name.capitalize()}")
print(f"   - Nugget: {best_model['nugget']:.2f} (variabilidad a escala micro)")
print(f"   - Sill: {best_model['sill']:.2f} (varianza total)")
print(f"   - Range: {best_model['range']:.0f} m (distancia de autocorrelación)")

print(f"\n2. INTERPOLACIÓN:")
print(f"   - Puntos de muestreo: {len(coords)}")
print(f"   - Puntos interpolados: {len(pred_coords)}")

print(f"\n3. VALIDACIÓN CRUZADA:")
better = 'Kriging' if cv_kriging['R2'] > cv_idw['R2'] else 'IDW'
print(f"   - Mejor método: {better}")
print(f"   - IDW R²: {cv_idw['R2']:.4f} | RMSE: {cv_idw['RMSE']:.3f}")
print(f"   - Kriging R²: {cv_kriging['R2']:.4f} | RMSE: {cv_kriging['RMSE']:.3f}")

print(f"\n4. INTERPRETACIÓN:")
print(f"   - El rango de {best_model['range']:.0f} m indica que la densidad")
print(f"     de edificios está espacialmente correlacionada hasta esa distancia.")
print(f"   - La varianza de Kriging proporciona una medida de incertidumbre")
print(f"     en las predicciones, útil para planificación territorial.")

In [ ]:
# Guardar resultados
pred_gdf.to_file(PROCESSED_PATH / 'interpolation_results.gpkg', driver='GPKG')
cv_results.to_csv('../outputs/reports/cross_validation_summary.csv', index=False)

print("\nResultados guardados:")
print(f"  - {PROCESSED_PATH / 'interpolation_results.gpkg'}")
print(f"  - ../outputs/reports/cross_validation_summary.csv")